# Exotic Derivatives Trading Desk Analysis

**As of 16 May 2025**, we will:
1. Fetch market data  
2. Instantiate each of our four option classes  
3. Price & (where available) compute delta  
4. Summarize hedges for the combined portfolio  


In [2]:
# Cell 1: install dependencies (run once)
!pip install --quiet yfinance



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
# Cell 2: imports

import datetime
import numpy as np
import yfinance as yf

from derivatives import (
    EuropeanCall,
    AmericanPut,
    UpAndInBarrierCall,
    BasketCall
)


In [8]:
# Cell 3: Market inputs as of 2025-05-16
import datetime
import yfinance as yf

# Define trade date and next day (end is exclusive in yf.download)
trade_date = "2025-05-16"
next_day  = (
    datetime.datetime.strptime(trade_date, "%Y-%m-%d")
    + datetime.timedelta(days=1)
).strftime("%Y-%m-%d")

tickers = ["BHP.AX", "CBA.AX", "WES.AX", "CSL.AX", "WDS.AX", "MQG.AX"]

# Download the two-day window and grab the 16th
df = yf.download(tickers, start=trade_date, end=next_day)["Close"]
S0 = df.loc[trade_date].to_dict()

# Example output to check
print("Spot prices as of", trade_date)
for t, price in S0.items():
    print(f"  {t}: {price:.2f}")

# Implied vols (placeholders for now)
vol = {
    "BHP.AX": 0.25,
    "CBA.AX": 0.22,
    "WES.AX": 0.30,
    "CSL.AX": 0.20,
    "WDS.AX": 0.28,
    "MQG.AX": 0.23
}

# Flat risk-free rate
r = 0.04  # 4%

print("Vols:", vol)
print("r:", r)


[                       0%                       ]

[*********************100%***********************]  6 of 6 completed

Spot prices as of 2025-05-16
  BHP.AX: 39.72
  CBA.AX: 169.66
  CSL.AX: 241.82
  MQG.AX: 207.35
  WDS.AX: 21.92
  WES.AX: 82.56
Vols: {'BHP.AX': 0.25, 'CBA.AX': 0.22, 'WES.AX': 0.3, 'CSL.AX': 0.2, 'WDS.AX': 0.28, 'MQG.AX': 0.23}
r: 0.04


In [9]:
# Cell 4: Instantiate & price each option

import datetime
from derivatives import EuropeanCall, AmericanPut, UpAndInBarrierCall, BasketCall

# 1) European call on BHP (expiry 15 Sept 2027, strike = 98% spot)
T1 = (datetime.datetime(2027, 9, 15) - datetime.datetime(2025, 5, 16)).days / 365
bhp_call = EuropeanCall(
    S0   = S0["BHP.AX"],
    K    = 0.98 * S0["BHP.AX"],
    T    = T1,
    r    = r,
    sigma= vol["BHP.AX"]
)
price_bhp = bhp_call.price()
delta_bhp = bhp_call.delta()

# 2) American put on CBA (expiry 15 May 2026, strike = 170)
T2 = (datetime.datetime(2026, 5, 15) - datetime.datetime(2025, 5, 16)).days / 365
cba_put = AmericanPut(
    S0   = S0["CBA.AX"],
    K    = 170.0,
    T    = T2,
    r    = r,
    sigma= vol["CBA.AX"]
)
price_cba = cba_put.price(steps=500)

# 3) Up-and-in barrier call on WES (expiry same as BHP, strike=80, barrier=100)
wes_barrier = UpAndInBarrierCall(
    S0      = S0["WES.AX"],
    K       = 80.0,
    T       = T1,
    r       = r,
    sigma   = vol["WES.AX"],
    barrier = 100.0
)
price_wes = wes_barrier.price(paths=10000, steps=252)

# 4) European basket call on [BHP, CSL, WDS, MQG] (expiry 17 July 2025, strike=175)
T3 = (datetime.datetime(2025, 7, 17) - datetime.datetime(2025, 5, 16)).days / 365
tickers_basket = ["BHP.AX", "CSL.AX", "WDS.AX", "MQG.AX"]
S_list    = [S0[t] for t in tickers_basket]
sig_list  = [vol[t] for t in tickers_basket]
weights   = [0.10, 0.35, 0.15, 0.40]

basket = BasketCall(
    S0_list    = S_list,
    weights    = weights,
    K          = 175.0,
    T          = T3,
    r          = r,
    sigma_list = sig_list
)
price_basket = basket.price(paths=10000, steps=252)

# Print results
print(f"BHP European Call  → Price: {price_bhp:.2f}, Delta: {delta_bhp:.4f}")
print(f"CBA American Put   → Price: {price_cba:.2f}")
print(f"WES Barrier Call   → Price: {price_wes:.2f}")
print(f"Basket Call        → Price: {price_basket:.2f}")

BHP European Call  → Price: 8.07, Delta: 0.6873
CBA American Put   → Price: 169.99
WES Barrier Call   → Price: 19.57
Basket Call        → Price: 0.20


## Pricing & Hedge Summary

Below are the prices and deltas (where available).


In [10]:
# Cell 5: Summary of prices & deltas
import pandas as pd

summary = pd.DataFrame({
    "Instrument": [
        "BHP European Call",
        "CBA American Put",
        "WES Up-and-In Barrier Call",
        "Four-Stock Basket Call"
    ],
    "Price": [
        price_bhp,
        price_cba,
        price_wes,
        price_basket
    ],
    "Delta": [
        delta_bhp,
        None,     # we’ll compute this next
        None,     # and this
        None      # and this
    ]
})

display(summary)


,Instrument,Price,Delta
0,BHP European Call,8.071603,0.687339
1,CBA American Put,169.990826,NaN
2,WES Up-and-In Barrier Call,19.566606,NaN
3,Four-Stock Basket Call,0.196322,NaN


from here getting a bit lost in the sauce

In [11]:
# Cell 6: Finite-difference deltas

eps_cba = 0.01 * S0["CBA.AX"]  # 1% bump
put_up   = AmericanPut(S0["CBA.AX"]+eps_cba, 170, T2, r, vol["CBA.AX"]).price(steps=500)
put_down = AmericanPut(S0["CBA.AX"]-eps_cba, 170, T2, r, vol["CBA.AX"]).price(steps=500)
delta_cba = (put_up - put_down) / (2*eps_cba)

eps_wes = 0.01 * S0["WES.AX"]
bic_up   = UpAndInBarrierCall(S0["WES.AX"]+eps_wes, 80, T1, r, vol["WES.AX"], barrier=100).price(paths=5000)
bic_down = UpAndInBarrierCall(S0["WES.AX"]-eps_wes, 80, T1, r, vol["WES.AX"], barrier=100).price(paths=5000)
delta_wes = (bic_up - bic_down) / (2*eps_wes)

# For the basket, bump each underlying one at a time to get per-stock deltas:
delta_basket = {}
for i, ticker in enumerate(["BHP.AX","CSL.AX","WDS.AX","MQG.AX"]):
    eps_i = 0.01 * S0[ticker]
    S_up = S_list.copy(); S_up[i] += eps_i
    S_dn = S_list.copy(); S_dn[i] -= eps_i
    bc_up = BasketCall(S_up, weights, 175, T3, r, sig_list).price(paths=5000)
    bc_dn = BasketCall(S_dn, weights, 175, T3, r, sig_list).price(paths=5000)
    delta_basket[ticker] = (bc_up - bc_dn) / (2*eps_i)

# Update the summary table
summary.loc[1, "Delta"] = delta_cba
summary.loc[2, "Delta"] = delta_wes
summary.loc[3, "Delta"] = sum(weights[i] * delta_basket[t] 
                                for i, t in enumerate(["BHP.AX","CSL.AX","WDS.AX","MQG.AX"]))

display(summary)
print("\nBasket deltas by stock:", delta_basket)


,Instrument,Price,Delta
0,BHP European Call,8.071603,0.687339
1,CBA American Put,169.990826,-0.000054
2,WES Up-and-In Barrier Call,19.566606,1.223106
3,Four-Stock Basket Call,0.196322,0.131951



Basket deltas by stock: {'BHP.AX': np.float64(0.03215759097413797), 'CSL.AX': np.float64(0.1465066966928023), 'WDS.AX': np.float64(0.0659140661883176), 'MQG.AX': np.float64(0.16892784087184837)}


In [12]:
# Cell 7: Compute hedge quantities (assume 1-option = 1-share notional)
summary["Hedge Qty"] = -summary["Delta"]  # negative sign to offset exposure
display(summary)


,Instrument,Price,Delta,Hedge Qty
0,BHP European Call,8.071603,0.687339,-0.687339
1,CBA American Put,169.990826,-0.000054,0.000054
2,WES Up-and-In Barrier Call,19.566606,1.223106,-1.223106
3,Four-Stock Basket Call,0.196322,0.131951,-0.131951


## Portfolio Hedge Summary

As of 16 May 2025, our four‐instrument portfolio is priced and hedged as follows:

| Instrument                         | Price     | Δ       | Hedge Qty            |
|------------------------------------|----------:|--------:|----------------------|
| **BHP European Call**              | \$8.07    | 0.6873  | Sell 0.6873 shares BHP  |
| **CBA American Put**               | \$169.99  | –0.00005| Buy 0.00005 shares CBA |
| **WES Up-and-In Barrier Call**     | \$19.57   | 1.2231  | Sell 1.2231 shares WES  |
| **Four-Stock Basket Call**         | \$0.1963  | 0.1320  | Sell the basket as:   |
| • BHP (10%)                        |           |         | 0.01320 shares BHP     |
| • CSL (35%)                        |           |         | 0.04618 shares CSL     |
| • WDS (15%)                        |           |         | 0.01979 shares WDS     |
| • MQG (40%)                        |           |         | 0.05278 shares MQG     |

---

**Interpretation:**

- **BHP Call**: You’re long 0.6873 Δ in BHP per call, so you’d **sell** ~0.69 shares of BHP to hedge each call contract.  
- **CBA Put**: Δ≈0, so hedging is negligible.  
- **WES Barrier Call**: You’re long 1.2231 Δ in WES per barrier call—**sell** ~1.22 shares of WES per contract.  
- **Basket Call**: Total Δ=0.1320 per basket; implement by trading each underlying in the weights shown above.

---

### Next Steps & Enhancements

1. **Greeks Expansion**: Add vega & theta for more comprehensive risk metrics.  
2. **Curve Boot-strapping**: Replace the flat 4% rate with a full discount-curve from OIS/swap data.  
3. **Unit Testing**: Write pytest routines to verify each class against benchmark values.  
4. **Performance Tuning**: Increase Monte Carlo paths or vectorize loops for greater precision.  
5. **Documentation**: Flesh out markdown narratives to justify model choices and data sources.

This completes the end-to-end MVP: market data → pricing → delta hedging → clear trade instructions.  
